# Nesystematické prohledávání stavového prostoru
## Symetrická úloha obchodního cestujícího (Symetric Travelling Salesman Problem, TSP)
Je dána množina měst W a matice C vzdáleností mezi dvojicemi měst z W. Úkolem obchodního cestujícího je projít těmito městy a následně se vrátit do výchozího města s minimálními výdaji na cestu a zároveň navštívit každé město právě jednou.

Předpokládejte, že úloha je formulována v Euklidovském prostoru, matice C vzdáleností mezi městy je symetrická, tj. platí c(i,j) = c(j,i) (tj. cestující může mezi městy cestovat oběma směry), a platí trojúhelníková nerovnost c(i,k) < c(i,j) + c(j,k) pro všechna i, j, k z W.

S využitím aparátu teorie grafů může být úloha formulována následovně: nechť G = (V,A) je neorientovaný graf. V označuje množinu vrcholů grafu, tyto vrcholy reprezentují města. A je množina hran s nezápornými ohodnoceními, která odpovídají prvkům matice C vzdáleností mezi městy. Úkolem je v daném grafu nalézt nejkratší kružnici, která prochází všemi vrcholy právě jednou (Hamiltonovské kružnice).
## Zadání
### TSP pomocí algoritmu Hill-Climbing
Navrhněte a implementujte řešení problému obchodního cestujícíco pomocí hill-climbing algoritmu.

#### Dílčí úkoly:

* Navrhněte a implementujte vhodné kódování problému TSP resp. N-královen pro řešení pomocí hill-climbing algoritmu
* Navrhněte účelovou funkci
* Definujte různé varianty okolí daného stavu
* Implementujte metodu na enumeraci stavů v definovaném okolí
* Implementujte nějakou metodu na únik z lokálního extrému například restarty
* Pro cvičení je připravena šablona.

Použití šablony není vyžadováno. Rozhodnete-li se implementovat algoritmus bez šablony, zajistěte vizualizaci průběhu algoritmu v terminálu.

In [1]:
from csv import reader
import copy
import random

## Nastavení
* Nastavte si zde prosím všechny potřebné hodnoty pro hill climbing.
* **fileName** = jméno souboru, ze kterého chcete načíst data
* **start** = startovní město
* **neighboursVariant** = varianta výběru sousedů, může být 1 nebo 2 
    * ***1*** Jako sousedy vybere všechny stavy, které jsou odlišné od původního, pouze prohozením vždy dvou, ne nutně sousedních, měst.
    * ***2*** Jako sousedy vybere všechny stavy, které jsou odlišné od původního, pouze prohozením vždy jedné dvojice sousedních čísel.
* **maxIter** = maximální počet iterací, které má provést Hill Climbing.
* **maxRestarts** = maximální počet restartů algoritmu Hill Climbing ve funkci escapeLocalExtreme, která se snaží zabránít uváznutí algoritmu v lokálním extrému.

In [2]:
# Zde prosím přepište proměnné pokud chcete jiné hodnoty pro výpočet
fileName = 'distances-10.csv'
start = 'Praha'
neighboursVariant = 2
maxIter = 1000
maxRestarts = 1000

## Pomocné funkce

### Data from csv file into list of lists

In [3]:
def readFile(fileName):
    with open(fileName, 'r') as read_distances_10:
        csv_reader = reader(read_distances_10)
        return list(csv_reader)

In [4]:
# Ukázka
file = readFile('distances-10.csv')
print(str(file))

### Make variables cities, distances
* cities = List měst
* distances = List of List with distances between cities

In [5]:
def makeVar(distances_smt):
    distances = copy.deepcopy(distances_smt)
    cities = [i.pop(0) for i in distances] 
    return cities, distances

In [6]:
# Ukázka
cities, distances = makeVar(file)
print(distances)
print(cities)

### Délka cesty - účelová funkce

In [7]:
def routeLength(distance, cities, solution):
    routeLength = 0
    for i in range(1, len(solution)):
        routeLength += int(distances[cities.index(solution[i - 1])][cities.index(solution[i])])
    return routeLength
    

### Random first solution

In [8]:
def randomState(cities, start):
    solution = [start]
    cities_solution = copy.deepcopy(cities)
    cities_solution.remove(start)
    for i in range(len(cities) - 1):
        randomCity = cities_solution[random.randint(0, len(cities_solution) - 1)]
        solution.append(randomCity)
        cities_solution.remove(randomCity)
    solution.append(start)
    return solution

In [9]:
# Ukázka
print(randomState(cities, 'Brno'))

['Brno', 'Rijeka', 'Poreč', 'Pula', 'Karlovac', 'Makarská', 'Plitv.jez', 'Dubrovník', 'Praha', 'Lublaň', 'Brno']


In [10]:
# Ukázka
routeLength(distances, cities, randomState(cities, 'Brno'))

5331

### Neighbours of a solution
* Sousední řešení je takové platné řešení, které je pouze málo jiné odlišné vůči součastnému řešení.
* 1 Jako sousedy vybere všechny stavy, které jsou odlišné od původního, pouze prohozením vždy dvou, ne nutně sousedních, měst.
* 2 Jako sousedy vybere všechny stavy, které jsou odlišné od původního, pouze prohozením vždy jedné dvojice sousedních čísel.

In [11]:
def getNeighbours_1(solution):
    neighbours = []
    for i in range(1, len(solution) - 1):
        for j in range(i + 1, len(solution) - 1):
            neighbour = copy.deepcopy(solution)
            neighbour[i] = solution[j]
            neighbour[j] = solution[i]
            neighbours.append(neighbour)
    return neighbours

In [12]:
def getNeighbours_2(solution):
    neighbours = []
    for i in range(1, len(solution) - 2):
        neighbour = copy.deepcopy(solution)
        neighbour[i] = solution[i + 1]
        neighbour[i + 1] = solution[i]
        neighbours.append(neighbour)
    return neighbours

In [13]:
def getNeighbours(solution, variant):
    if variant == 1:
        return getNeighbours_1(solution)
    else:
        return getNeighbours_2(solution)

In [14]:
# Ukázka
solutionExample = randomState(cities, cities[0])
print(solutionExample)

['Brno', 'Makarská', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno']


In [15]:
# Ukázka
neighbours_1 = getNeighbours_1(solutionExample)
print(str(neighbours_1))

[['Brno', 'Poreč', 'Makarská', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Pula', 'Poreč', 'Makarská', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Praha', 'Poreč', 'Pula', 'Makarská', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Rijeka', 'Poreč', 'Pula', 'Praha', 'Makarská', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Dubrovník', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Makarská', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Lublaň', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Makarská', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Plitv.jez', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Makarská', 'Karlovac', 'Brno'], ['Brno', 'Karlovac', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Makarská', 'Brno'], ['Brno', 'Makarská', 'Pula', 'Poreč', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.

In [16]:
# Ukázka
neighbours_2 = getNeighbours_2(solutionExample)
print(str(neighbours_2))

[['Brno', 'Poreč', 'Makarská', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Pula', 'Poreč', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Praha', 'Pula', 'Rijeka', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Pula', 'Rijeka', 'Praha', 'Dubrovník', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Pula', 'Praha', 'Dubrovník', 'Rijeka', 'Lublaň', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Lublaň', 'Dubrovník', 'Plitv.jez', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Plitv.jez', 'Lublaň', 'Karlovac', 'Brno'], ['Brno', 'Makarská', 'Poreč', 'Pula', 'Praha', 'Rijeka', 'Dubrovník', 'Lublaň', 'Karlovac', 'Plitv.jez', 'Brno']]


### Enumerace sousedních stavů a hledání nejlepšího souseda

In [17]:
def getBestNeighbour(distances, neighbours):
    bestRouteLength = routeLength(distances, cities, neighbours[0])
    bestNeighbour = neighbours[0]
    for neighbour in neighbours:
        currentRouteLength = routeLength(distances, cities, neighbour)
        if currentRouteLength < bestRouteLength:
            bestRouteLength = currentRouteLength
            bestNeighbour = neighbour
    return bestNeighbour, bestRouteLength

### Hill Climbing

In [18]:
def hillClimbing(start, cities, distances, maxIters, neighboursVariant):
    firstState = randomState(cities, start)
    firstStateLength = routeLength(distances, cities, firstState)
    neighbours = getNeighbours(firstState, neighboursVariant)
    bestNeighbour, bestNeighbourLength = getBestNeighbour(distances, neighbours)
    
    i = 0
    currentState = firstState
    currentStateLenght = firstStateLength
    while i < maxIters and  bestNeighbourLength < currentStateLenght:
        i += 1
        currentState = bestNeighbour
        currentStateLenght = bestNeighbourLength
        """ print(str(currentState) + ',\n' + str(currentStateLenght)) """
        neighbours = getNeighbours(currentState, neighboursVariant)
        bestNeighbour, bestNeighbourLength = getBestNeighbour(distances, neighbours)
    
    return currentState, currentStateLenght

In [19]:
# Ukázka
print(hillClimbing('Brno', cities, distances, 100, 1))

(['Brno', 'Praha', 'Karlovac', 'Plitv.jez', 'Makarská', 'Dubrovník', 'Poreč', 'Pula', 'Rijeka', 'Lublaň', 'Brno'], 2817)


### Únik z lokálního extrému

In [20]:
def escapeLocalExtreme(start, cities, distances, maxIter=100, maxRestarts=100, neighboursVariant=1):
    bestState = randomState(cities, start)
    bestLength = routeLength(distances, cities, bestState)
    
    i = 0
    while i < maxRestarts:
        solution, solutionLength = hillClimbing(start, cities, distances, maxIter, neighboursVariant)
        if solutionLength < bestLength:
            bestState = solution
            print(bestState)
            bestLength = solutionLength
            print(bestLength)
        i += 1

In [21]:
# Ukázka
escapeLocalExtreme('Brno', cities, distances, 1000, 1000)

['Brno', 'Praha', 'Karlovac', 'Lublaň', 'Poreč', 'Pula', 'Rijeka', 'Makarská', 'Dubrovník', 'Plitv.jez', 'Brno']
2966
['Brno', 'Praha', 'Lublaň', 'Poreč', 'Pula', 'Rijeka', 'Makarská', 'Dubrovník', 'Plitv.jez', 'Karlovac', 'Brno']
2795


## Celý program s vašemi daty

In [22]:
def program(fileName, start, maxIter, maxRestarts, neighboursVariant):
    file = readFile(fileName)
    cities, distances = makeVar(file)
    escapeLocalExtreme(start, cities, distances, maxIter, maxRestarts, neighboursVariant)

In [23]:
program(fileName, start, maxIter, maxRestarts, neighboursVariant)

['Praha', 'Brno', 'Pula', 'Poreč', 'Dubrovník', 'Makarská', 'Plitv.jez', 'Karlovac', 'Rijeka', 'Lublaň', 'Praha']
3015
['Praha', 'Lublaň', 'Rijeka', 'Pula', 'Poreč', 'Karlovac', 'Dubrovník', 'Makarská', 'Plitv.jez', 'Brno', 'Praha']
2988
['Praha', 'Lublaň', 'Poreč', 'Pula', 'Rijeka', 'Makarská', 'Dubrovník', 'Plitv.jez', 'Karlovac', 'Brno', 'Praha']
2795
